# Chat Bot con Deep Learning y NLP

## Instalaciones

In [ ]:
!pip install nltk
nltk.download('punkt')
!python -m spacy download es

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
     |████████████████████████████████| 16.2 MB 1.4 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/es_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/es
You can now load the model via spacy.load('es')


Importaciones

In [ ]:
import nltk
from nltk.stem import SnowballStemmer
import spacy
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import random
import json

## Definicion de métodos para usar en el pipeline de procesamiento de texto

Incializacion de instancias

In [ ]:
stemmer = SnowballStemmer('spanish')
nlp_spacy = spacy.load('es')

Limpieza de texto

In [ ]:
def limpiar(texto):
  import re
  texto = re.sub(r'[^ \w+]', '', texto).strip()
  texto = re.sub("\s\s+" , " ", texto).strip()
  return texto.lower()

Tokenización

In [ ]:
def tokenizar(frase,nlp_spacy=nlp_spacy):
  doc = nlp_spacy(frase)
  tokens = [token.text for token in doc]
  return tokens

Stemming

In [ ]:
def stem(palabra,stemmer=stemmer):
  return stemmer.stem(palabra)

Bolsa de palabras

In [ ]:
def bow(frase_tokenizada,lista_palabras):
  """
  sentence = ["hello","how","are","you"]
  words = ["hi","hell","I","you","bye","thank","cool"]
  bwg = [   0,     1,    0,   1,    0,     0,     0]
  """
  stemmed = [stem(w) for w in frase_tokenizada] # Realizamos el stem sobre la frase
  bag = np.zeros(len(lista_palabras), dtype=np.float32) # Vector con todo 0s de la longitud de el diccionario
  for ind, palabra in enumerate(lista_palabras): # Para toda la lista de palabras
    if palabra in stemmed: # Si la palabra del diccionario aparece en la frase
      bag[ind] = 1.0 # Pongo el inidice a 1
  return bag

# Carga de datos de entrenamiento

Acceso a Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
PATH_DATA= '/content/drive/MyDrive/nlp_introduction_course/data'
MODELS_DATA= '/content/drive/MyDrive/Colab Notebooks/nlp_introduction_course/models'
FILE_NAME = 'intents.json'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Leer los intents

In [ ]:
import json
with open(f'{PATH_DATA}/{FILE_NAME}','r') as f:
  intents = json.load(f)
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hola', 'Hey', '¿Como estas?', '¿Hay alguien ahí?', 'Buenas', 'Buenos días'], 'responses': ['Hola', 'Hola, gracias por saludar', 'Hola, ¿En que puedo ayudarte?', 'Hola, ¿Que puedo hacer por ti?']}, {'tag': 'goodbye', 'patterns': ['Adios', 'Hasta luego', 'Bye Bye', 'Chao'], 'responses': ['Te veo despues, gracias', 'Que tengas un buen día', '¡Adios! Espero verte pronto.']}, {'tag': 'thanks', 'patterns': ['Gracias', 'Muchas gracias', 'Me has ayudado', 'gracias por ayudarme'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'items', 'patterns': ['¿Que articulos tienes?', '¿Que vendes?', '¿Que puedes venderme?', '¿Que puedo comprar?'], 'responses': ['Vendo cofe y té', 'Puedo vender cafe y té']}, {'tag': 'payments', 'patterns': ['¿Que metodos de pago aceptas?', '¿Aceptas tarjeta de credito?', '¿Puedo pagar con tarjeta de credito?', '¿Aceptas Visa?', '¿Puedo pagar con Visa?', '¿Aceptas Mastercar?', '¿Puedo pagar con Mastercar

Vamos a crear una fucion para aplicar Limpieza, tokenizacion y Stemming y obtener la lista de palabras, la lista de tags, y los conjuntos de entrenamiento en forma de tuplas (tokens, tag)

In [ ]:
def preparar_datos(intents):
  lista_palabras = [] # Lista de palabras
  tags = [] # Intenciones
  xy = [] # Conjunto de entrenamiento en forma de tuplas (tokens, tag)
  for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag) # Añadimos el contenido de tag
    for patron in intent['patterns']: # Por cada patron
      patron = limpiar(patron) # Limpio pasando a minusculas, quitando signos de puntuación y espacios
      tokens = tokenizar(patron) # Tokenizo y obtengo la lista de palabras
      tokens_stemmed = [stem(t) for t in tokens]
      lista_palabras.extend(tokens_stemmed) # Añado la lista de palabras
      xy.append((tokens, tag)) # Para el conjunto de entrenamiento añado la tupla forma de tuplas (tokens, tag)
  lista_palabras = sorted(set(lista_palabras))# Dejamos unicamente las palabras únicas, y las ponemos de forma ordenada
  tags = sorted(set(tags)) # Ordenamos los tags
  x_train = []
  y_train = []
  for (tokens,tag) in xy:
    bag = bow(tokens,lista_palabras) # Calculamos la bolsa de palabras de la frase
    x_train.append(bag) # Lo añadimos a la lista de valores de entrenamiento

    label = tags.index(tag) # Obtenemos el indice de la label
    y_train.append(label) # Lo añadimos a los valores a predecir
  
  return lista_palabras,tags,np.array(x_train),np.array(y_train) # Retornamos lista_palabras, tags, x_train y y_train

In [ ]:
lista_palabras,tags,x_train,y_train = preparar_datos(intents)

## Creamos objeto para gestionar el dataset

In [ ]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

Hiperparametros

In [ ]:
batch_size = 8

In [ ]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

### Modelo PyTorch y entrenamiento

Creamos una clase para el modelo

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

La instanciamos

In [ ]:
# Hyper-parameters 
num_epochs = 2000
batch_size = 8
learning_rate = 0.001
input_size = len(lista_palabras)
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

56 7


Bucle de entrenamiento

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Usamos si podemos CUDA si no cpu
model = NeuralNet(input_size, hidden_size, output_size).to(device) # Inicializamos el modelo
criterion = nn.CrossEntropyLoss() # Funcion de perdida CrossEntropyLoss
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # Optimizador Adam
for epoch in range(num_epochs): # Para todas las epocas
    for (words, labels) in train_loader: # Datos de entrenamiento
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    if loss.item() < 0.0001:
      break


print(f'final loss: {loss.item():.6f}')

Epoch [100/2000], Loss: 0.5372
Epoch [200/2000], Loss: 0.2025
Epoch [300/2000], Loss: 0.0080
Epoch [400/2000], Loss: 0.0131
Epoch [500/2000], Loss: 0.0012
Epoch [600/2000], Loss: 0.0011
Epoch [700/2000], Loss: 0.0021
Epoch [800/2000], Loss: 0.0001
final loss: 0.000052


Guardamos el modelo

In [ ]:
data = {
  "model_state": model.state_dict(),
  "input_size": input_size,
  "hidden_size": hidden_size,
  "output_size": output_size,
  "all_words": lista_palabras,
  "tags": tags
}

In [ ]:
MODEL_FILE = "chatbot_data.pth"
torch.save(data, f'{MODELS_DATA}/{MODEL_FILE}')

print(f'training complete. file saved to {MODELS_DATA}/{MODEL_FILE}')

training complete. file saved to /content/drive/MyDrive/Colab Notebooks/nlp_introduction_course/models/chatbot_data.pth


# Chatbot

Cargamos el modelo

In [ ]:
data = torch.load(f'{MODELS_DATA}/{MODEL_FILE}')

Actualizamos los parametros desde data

In [ ]:
input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

Cargamos los intents

In [ ]:
with open(f'{PATH_DATA}/{FILE_NAME}', 'r') as json_data:
    intents = json.load(json_data)

Instanciamos el modelo desde el diccionario con los pesos de la red neuronal

In [ ]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()


NeuralNet(
  (l1): Linear(in_features=56, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=7, bias=True)
  (relu): ReLU()
)

Bucle infinito para el Chat

In [ ]:
bot_name = "Bot"
print(f"{bot_name}: Vamos a chatear! (escribe 'salir' para salir)")
while True:
    sentence = input("You: ")
    if sentence == "salir":
        print('...Saliendo XD')
        break

    sentence = tokenizar(sentence)
    X = bow(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: Perdona no te he entendido...")

Bot: Vamos a chatear! (escribe 'salir' para salir)
You: salir
...Saliendo XD
